# 互评作业2：频繁模式与关联规则

运行此notebook需要使用matplotlib库。

## 0 数据集简介及任务说明

本次互评作业使用的数据集是Chicago Building Violations。本数据集包含芝加哥从2006年到2019年所开具的建筑违规罚单。这些罚单记录了罚单的ID、产生日期、最后修改日期、违规的具体情形、建筑地址等等信息。数据集中记录了每一列的意义，如下所示：

In [ ]:
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict

fjson = open("Chicago Building Violations/socrata_metadata.json")

json_data=json.load(fjson)
print('#\tcolumn')
for column in json_data['columns']:
    print(column['position']-1,end='\t')
    print(column['name'])
    if 'description' in column:
        print('\t\t'+column['description'])
        
# 数据集有些大，进行一些清理工作以节约内存
del json_data
fjson.close()

接下来介绍我们讨论的主要问题。我们先统计记录的个数以及涉及房屋的数量：

In [ ]:
def read_csv(filename):
    with open(filename) as f:
        csv_content = csv.reader(f)
        columns = next(csv_content)
        rows = [row for row in csv_content]
    return columns, rows

# 对标称值计数以及画条形图
def count_occurence(data,column,ignore_empty=True):
    '''统计数据某个特定列中所有元素的出现次数，返回一个dict-like
    如果置ignore-empty为True，则返回的字典中不包含空值的计数'''
    count = defaultdict(int)
    for row in data:
        if ignore_empty and row[column]=='':
            continue
        count[row[column]]+=1
    return count

cols_csv,rows_csv = read_csv('Chicago Building Violations/building-violations.csv')
print(f'数据集中有{len(rows_csv)}条记录')
addr_occurence=count_occurence(rows_csv,16,False)
assert '' not in addr_occurence # 显然，数据集中每一条记录都是有对应房屋地址的
print(f'数据集涉及{len(addr_occurence)}个房屋地址')

可见数据集中涉及房屋地址的数量比罚单数量少一个数量级左右，因此本次互评作业的任务是探究房屋开具罚单（确切地说，罚单类型）的模式和关联关系。

## 1 数据的预处理

接下来，我们对数据集进行一些预处理，仅保留我们需要的内容。我们想要保留的数据只有两类：1）违规代码与其解释的对应关系；2）每个房屋产生过哪些违规罚单。为了节约内存，我们忽略掉其他部分。

In [ ]:
# 违规代码到对应解释的映射关系
viol2code=[]
viol2desp=[]
code2viol={}
# 建筑物到其地址的关系
bld2addr=[]
addr2bld={}
# 建筑物ID到其产生的罚单表的关系
bld2viols=[]
for row in rows_csv:
    # 整理违规代码对应的解释
    code = row[3]
    desp = row[6]
    violid=-1
    if code in code2viol:
        violid=code2viol[code]
    else:
        violid = len(viol2code)
        viol2code.append(code)
        viol2desp.append(desp)
        code2viol[code]=violid
    # 整理建筑物罚单
    addr = row[16]
    bldid = -1
    if addr in addr2bld:
        bldid = addr2bld[addr]
    else:
        bldid = len(bld2addr)
        bld2addr.append(addr)
        bld2viols.append([])
        addr2bld[addr]=bldid
    bld2viols[bldid].append(violid)
# 原始数据量很大，故删除之以节省内存
del rows_csv
del addr2bld
del code2viol

我们来看一下每个建筑物对应的罚单数量的分布情况：

In [ ]:
viol_count=sorted([len(l)for l in bld2violations])
bld_count = len(viol_count)
plt.figure()
plt.title('Q-chart on violations-per-building')
plt.plot(viol_count,[i/bld_count for i in range(bld_count)])
plt.show()

从上述统计看，绝大多数建筑只有很少（不到50个）的罚单，但个别建筑有很多罚单。

## 2 频繁模式统计

现在我们统计这对关系中的频繁模式。我们使用Apriori方法找到所有的频繁模式：

In [ ]:
min_support=.01

#Apriori算法的原实现来自
#https://blog.csdn.net/qq_39872846/article/details/105291265
#使用时有改动

# 求第一次扫描数据库后的候选集
# 就是求这个数据库中出现了几个元素，然后返回
def item(dataset):      
    c1 = [] # 存放候选集元素

    for x in dataset:
        for y in x:
            if [y] not in c1:
                c1.append( [y] )
    c1.sort()
    return c1

# 求第k次候选集
def get_candidate(Fk, K):       
    ck = []    #存放产生候选集

    for i in range(len(Fk)):
        for j in range(i+1, len(Fk)):
            L1 = list(Fk[i])[:K-2]
            L2 = list(Fk[j])[:K-2]
            L1.sort()
            L2.sort() #先排序，在进行组合

            if L1 == L2:
                if K > 2:
                    new = list(set(Fk[i]) ^ set(Fk[j]) )
                else:
                    new = set()
                for x in Fk: 
                    #剪枝：new是 x 的子集，并且 还没有加入 ck 中
                    if set(new).issubset(set(x)) and list(set(Fk[i]) | set(Fk[j])) not in ck: 
                        ck.append( list(set(Fk[i]) | set(Fk[j])) )
    return ck

def get_frequent_item(dataset, c, min_support):
    cut_branch = {}     #用来存放所有项集的支持度的字典
    for x in c:
        for y in dataset:
            if set(x).issubset(set(y)):     #如果 x 不在 y中，就把对应元素后面加 1
                cut_branch[tuple(x)] = cut_branch.get(tuple(x), 0) + 1
                #cut_branch[y] = new_cand.get(y, 0)表示如果字典里面没有想要的关键词，就返回0

    Fk = []       #支持度大于最小支持度的项集，  即频繁项集
    sup_dataK = {}  #用来存放所有 频繁 项集的支持度的字典

    for i in cut_branch:
        if cut_branch[i] >= min_support:
            Fk.append( list(i))
            sup_dataK[i] = cut_branch[i]
    return Fk, sup_dataK

# 此方法返回一个三维列表，第0维是历次迭代产生的频繁项集，第1维是各个频繁项集，第2维是集合中的各个项
def Apriori(dataset, min_support = 2):
    c1 = item (dataset) #返回一个二维列表，里面的每一个一维列表，都是第一次候选集的元素
    f1, sup_1 = get_frequent_item(dataset, c1, min_support)       #求第一次候选集

    F = [f1]      #将第一次候选集产生的频繁项集放入 F ,以后每次扫描产生的所有频繁项集都放入里面
    sup_data = sup_1       #一个字典，里面存放所有产生的候选集，及其支持度

    K = 2 #从第二个开始循环求解，先求候选集，在求频繁项集

    while (len(F[K-2]) > 1):  #k-2是因为F是从0开始数的     #前一个的频繁项集个数在2个或2个以上，才继续循环，否则退出
        ck = get_candidate(F[K-2], K)  #求第k次候选集
        fk, sup_k = get_frequent_item(dataset, ck, min_support)     #求第k次频繁项集

        F.append(fk)    #把新产生的候选集加入F
        sup_data.update(sup_k)  #字典更新，加入新得出的数据
        K+=1
    return F, sup_data    #返回所有频繁项集， 以及存放频繁项集支持度的字典

